In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

from scipy.stats import chi2_contingency

### 데이터 불러오기

In [2]:
# 데이터를 불러온다.
df1 = pd.read_parquet('채널정보_전처리.parquet')

zero_after = [c for c in df1.columns if df1[c].eq(0).all()]
na_after   = [c for c in df1.columns if df1[c].isna().any()]

print("로드 후 0 전부 컬럼:", zero_after)
print("로드 후 NaN 있는 컬럼:", na_after)
print("로드 후 전체 컬럼 수:", len(df1.columns))

로드 후 0 전부 컬럼: []
로드 후 NaN 있는 컬럼: ['OS구분코드', 'Segment']
로드 후 전체 컬럼 수: 57


### Segment 분포 확인하기

In [3]:
# 1) Segment(타깃) 분포 살펴보기
print("▶ Segment 정보:")
print(df1['Segment'].value_counts())

print("\n▶ Segment 분포 및 비율 (%):")
print(df1['Segment'].value_counts(normalize=True).round(5))

▶ Segment 정보:
Segment
E    1922052
D     349242
C     127590
A        972
B        144
Name: count, dtype: int64

▶ Segment 분포 및 비율 (%):
Segment
E    0.80086
D    0.14552
C    0.05316
A    0.00040
B    0.00006
Name: proportion, dtype: float64


In [4]:
# One-Hot Encoding
df_ohe = pd.get_dummies(df1, columns=['Segment'], prefix='Seg')

# 결과 확인
print(df_ohe)

           기준년월            ID  인입일수_ARS_R6M  인입월수_ARS_R6M  인입후경과월_ARS  \
0        201807  TRAIN_000000             8             6           0   
1        201807  TRAIN_000001             0             0           0   
2        201807  TRAIN_000002             1             1           0   
3        201807  TRAIN_000003            10             6           0   
4        201807  TRAIN_000004             0             0           0   
...         ...           ...           ...           ...         ...   
2999995  201812    TEST_99995             0             0           0   
2999996  201812    TEST_99996             0             0           0   
2999997  201812    TEST_99997             0             0           0   
2999998  201812    TEST_99998             0             0           0   
2999999  201812    TEST_99999             0             0           0   

         인입횟수_ARS_B0M  이용메뉴건수_ARS_B0M  인입일수_ARS_B0M  방문월수_PC_R6M  \
0                   2               6             2    

### Segment와 변수 간 상관관계 분석
- 에타 제곱 분석의 신뢰도를 높이기 위해, 전체의 5% 미만을 차지하는 세그먼트는 표본 수가 부족하여 과도한 변동을 유발하고 해석에 혼란을 줄 수 있으므로 사전에 제외하였다.
- 에타 제곱 값이 0.01 미만인 변수는 그룹 간 차이를 설명하는 데 기여가 거의 없다고 판단하여 추가 분석 대상에서 제외하였다.

In [5]:
# Seg_로 시작하는 더미 컬럼 리스트 생성
dummy_cols = [c for c in df_ohe.columns if c.startswith('Seg_')]

# Segment 복원: 값이 1인 더미 컬럼에서 'Seg_' 접두사를 제거
df_ohe['Segment'] = df_ohe[dummy_cols].idxmax(axis=1).str.replace('Seg_', '')

# 수치형 변수만 남긴 df_num 생성 (더미 컬럼 제거)
df_num = df_ohe.drop(columns=dummy_cols).copy()

# 기존에 생성한 df_num을 df로 재할당
df = df_num

In [6]:
# 수치형 독립 변수와 범주형 타겟 변수 사이의 관련성 확인 (Eta Squared : 에타 제곱)
def eta_squared(y, x):
    # 제외할 세그먼트 리스트 설정
    drop_segs = ['A', 'B']
    
    # 제외 대상 세그먼트 필터 마스크 생성
    mask = ~y.isin(drop_segs)
    y_f = y[mask]
    x_f = x[mask]
    
    # 전체 x_f의 평균 산출하여 grand_mean에 저장
    grand_mean = x_f.mean()
    
    # 총 변동 SST 산출
    sst = ((x_f - grand_mean) ** 2).sum()
    
    # 필터링된 그룹별 x_f 리스트 생성
    groups = [x_f[y_f == val] for val in y_f.unique()]
    
    # 집단 간 변동 SSB 산출
    ssb = sum(len(g) * (g.mean() - grand_mean) ** 2 for g in groups)
    
    # Eta Squared 산출하여 반환
    return ssb / sst if sst != 0 else np.nan

In [7]:
# df = df_num 까지 수행한 뒤, 이 코드 블록을 실행

# 순서형 인코딩할 원본 _num 컬럼 리스트
ord_cols = [
    '인입횟수_ARS_R6M_num',
    '이용메뉴건수_ARS_R6M_num',
    '방문횟수_PC_R6M_num',
    '방문일수_PC_R6M_num',
    '방문횟수_앱_R6M_num'
]

# 순서형 인덱스를 매핑하여 df에 새로운 _ord 컬럼 생성
for col in ord_cols:
    unique_vals = sorted(df[col].unique())
    mapping     = {val: idx + 1 for idx, val in enumerate(unique_vals)}
    df[f'{col}_ord'] = df[col].map(mapping)

# 원본 _num 컬럼 삭제
df.drop(columns=ord_cols, inplace=True)


group_cols = [
    '인입횟수_ARS_R6M_num_ord',
    '이용메뉴건수_ARS_R6M_num_ord',
    '방문횟수_PC_R6M_num_ord',
    '방문일수_PC_R6M_num_ord',
    '방문횟수_앱_R6M_num_ord'
]

### 에타제곱 값 구하기

In [8]:
# 계산에서 제외할 컬럼 목록 설정
exclude_cols = ['Segment', 'ID', '기준년월']

# 수치형 컬럼 리스트 생성
numeric_cols = [
    col for col in df.columns
    if col not in exclude_cols and pd.api.types.is_numeric_dtype(df[col])
]

# 에타 제곱 결과 저장 리스트 초기화
eta_results = []

# 각 수치형 컬럼에 대해 에타 제곱값 계산
for col in numeric_cols:
    
    # 에타 제곱값 구하기
    eta_val = eta_squared(df['Segment'], df[col])
    
    # 결과 리스트에 추가
    eta_results.append({
        'feature': col,
        'eta_squared': eta_val
    })

# 리스트를 데이터프레임으로 변환
eta_df = pd.DataFrame(eta_results)

# 에타 제곱값이 0.01 이상인 피처만 필터링
eta_df = eta_df[eta_df['eta_squared'] >= 0.01]

# 에타 제곱 기준 내림차순 정렬
eta_df = eta_df.sort_values('eta_squared', ascending=False)

# 필터링된 피처 출력
display(eta_df)

,feature,eta_squared
40,불만제기후경과월_R12M,0.031948
7,방문후경과월_PC_R6M,0.029933
44,홈페이지_금융건수_R6M,0.029287
10,방문후경과월_앱_R6M,0.027972
17,방문횟수_앱_B0M,0.026572
46,홈페이지_금융건수_R3M,0.026409
9,방문월수_앱_R6M,0.026281
18,방문일수_앱_B0M,0.026085
6,방문월수_PC_R6M,0.025849
0,인입일수_ARS_R6M,0.025267


### 크래머의 V 값 구하기

In [9]:
# 범주형 변수 간 연관도 측정 함수 (Cramér’s V)
def cramers_v(x, y):
    
    # 교차표(confusion matrix) 생성
    cm = pd.crosstab(x, y)
    
    # 카이제곱 통계량
    chi2 = chi2_contingency(cm)[0]
    n    = cm.sum().sum()                       # 전체 표본 수
    phi2 = chi2 / n
    r, k = cm.shape                            # 행(row), 열(column) 개수
    
    # 보정된 phi2 및 차원 보정
    phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corr    = r - ((r-1)**2)/(n-1)
    k_corr    = k - ((k-1)**2)/(n-1)
    # Cramér’s V
    return np.sqrt(phi2_corr / min((k_corr-1), (r_corr-1)))

# OS구분코드와 Segment 간 Cramér’s V 구하기
v_os = cramers_v(df['OS구분코드'], df['Segment'])

# 결과 출력
print(f"Cramér’s V (OS구분코드 vs Segment) = {v_os:.4f}")

Cramér’s V (OS구분코드 vs Segment) = 0.0281


## 에타 제곱(Eta²) 해석 기준
- **작은 효과 (Small)**: 0.01 ≤ η² < 0.06  
- **중간 효과 (Medium)**: 0.06 ≤ η² < 0.14  
- **큰 효과 (Large)**: η² ≥ 0.14  

---

## 크래머의 V(Cramér’s V) 해석 기준
- **작은 연관 (Small)**: V < 0.10  
- **중간 연관 (Medium)**: 0.10 ≤ V < 0.30  
- **큰 연관 (Large)**: V ≥ 0.30  